In [1]:
# import libraries in jupyter notebook environment
import openpyxl
import pandas as pd #pd is linked to module: pandas
import math
import numpy as np #np is linked to module: numpy
import os


# print the current working directory
!pwd


/Users/stephxtchan/ECO421/A1


In [2]:
# read the portfolio value excel file and store the values under the name xls
xls = pd.ExcelFile('/Users/stephxtchan/ECO421/A1/PortfolioValue2020Feb03-2022Jan31.xlsx')

# read the xls file and import the ValueList spreadsheet and store the values under the name value_list
value_list = pd.read_excel(xls, sheet_name = 'ValueList')


In [3]:
# store the portfolio value into variable named values
max_row=len(value_list) # find the total number of rows in ValueList
values=[] # creat an empty list called values
for row in range(0, max_row): # use for loop to add each portfolio value into the list "values"
    values.append(float(value_list.loc[row][1])) 

In [4]:
# Define a function that finds the sortedloss(k)

def sortedloss(N):  # N is the user input of Time Horizon in days
    
    # Define the variables needed in the calculation
    m = len(values) # let m be the total number of portfolio values
    n = math.floor((m-1)/N) # let n be the largest k such that n*N <= m-1

    
    # Define the portfolio scenario as scenerio, known as S(k)
    # S(k) = V(k*N), where as V(k*N) is the k*N-th portfolio value, k = 0, 1, 2,..., n
    # Base Scenario S(0) is the first portfolio value V(0)
    scenerio = []
    for k in range(0, n+1):
        scenerio.append(float(values[k*N]))
    
    # Define portfolio loss as loss
    # loss(k) = S(0) - S(k), k = 1, 2,..., n
    loss = []
    for k in range(1, n+1):
        loss.append(float(scenerio[0] - scenerio[k]))
    
    # Sort list "loss" in ascending order by using numpy, define as sortedloss
    sortedloss = np.sort(loss)
    
            
    return sortedloss

In [5]:
# Define a function that finds the VaR and print the results

def var(sortedloss, x): # x% is the user input of confidence Level
    
    m = len(values) # let m be the total number of portfolio values
    n = math.floor((m-1)/N) # let n be the largest k such that n*N <= m-1
    w = int((x/100) * n) # let w be floor x% * n 
    
    # Convert sortedloss into a list
    sortedloss = sortedloss.tolist()
    
    # Define x% N-day VaR = sortedloss(w)
    VaR = sortedloss[w - 1] # since the index of the w-th loss is w-1
    
    # add VaR to the list "sortedloss"
    sortedloss.append(VaR)

    # Write the sortedloss and VaR into the excel spreadsheet "Scenario"
    with pd.ExcelWriter('/Users/stephxtchan/ECO421/A1/PortfolioValue2020Feb03-2022Jan31.xlsx',
                        engine = "openpyxl", mode = "a", if_sheet_exists = "replace") as writer:
        pd.DataFrame(sortedloss).to_excel(writer, sheet_name = 'Scenario', index = False, header = False)
    
    return VaR
    

# Note that if my export method did not overwrite the original Scenario sheet, please update panda to the newest version
# Panda update code: !pip3 install --upgrade pandas 
# Please update panda before runing this and the next chunk

In [6]:
# Create user input of Time Horizon(N) in days
N = int(input('Enter Time Horizon N in days (put only interger values): '))
# Note that 0 < N < 10
if N > 10:
    print("Time Horizon input is not valid")
elif N < 0:
    print("Time Horizon input is not valid")
else:
    sortedloss(N)
    
# Create user input of Condfidence Level 
x = float(input('Enter Confidence Level x in percentage (do not put % sign): '))

# Note that 90% < x < 100%
if x > 100:
    print("Confidence Level input is not valid")
elif x < 90:
    print("Confidence Level input is not valid")
else:
    var(sortedloss(N), x)
    print("The " + str(x) + "% " + str(N) + "-Day VaR is " + str(var(sortedloss(N), x))) 


Enter Time Horizon N in days (put only interger values): 5
Enter Confidence Level x in percentage (do not put % sign): 95
The 95.0% 5-Day VaR is 11481.420000000002


##### 